# Preprocessing Only Workflow

In this workflow, CFRL takes in an offline trajectory and then preprocesses the offline trajectory 
using `SyntheticPreprocessor`. The final output of the workflow is the preprocessed (debiased) 
offline trajectory. This workflow is appropriate when the user does not want to train policies using 
CFRL. Instead, the user can take the preprocessed trajectory to train a counterfactually fair policy 
using another reinforcement learning library or application that better fits their needs.

We begin by importing the liberaries needed for this demonstration.

In [1]:
# Need this temporarily to import CFRL before it is officially published to PyPI
import sys
sys.path.append("E:/learning/university/MiSIL/CFRL Python Package/CFRL")

In [2]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from cfrl.reader import read_trajectory_from_dataframe, convert_trajectory_to_dataframe
from cfrl.preprocessor import SequentialPreprocessor
np.random.seed(10) # ensure reproducibility
torch.manual_seed(10) # ensure reproducibility

## Data Loading

In this demonstration, we use an offline trajectory generated from a `SyntheticEnvironment` using some pre-specified transition rules. Although it is actually synthesized, we treat it as if it is from some unknown environment for pedagogical convenience in this demonstration.

The trajectory contains 500 individuals (i.e. $N=500$) and 10 transitions (i.e. $T=10$). The actions are binary (0 or 1) and were sampled using a random policy that selects 0 or 1 randomly with equal probability. It is stored in a tabular format in a `.csv` file. The sensitive attribute variable is bivariate, stored in columns `z1` and `z2`. The legit values of the sensitive attribute are $[0, 0]$, $[1, 0]$, $[0, 1]$, and $[1, 1]$. The state variable is tri-variate, stored in columns `state1`, `state2`, and `state3`. The actions are stored in the column `action` and rewards in the column `reward`. The tabular data also includes an extra irrelevant column `timestamp`. 

We can load and view the tabular data.

In [3]:
trajectory = pd.read_csv('../data/sample_data_large_multi_2.csv')
trajectory

,Unnamed: 0,ID,timestamp,z1,z2,action,reward,state1,state2,state3
0,0,1.0,1.0,1.0,1.0,NaN,NaN,4.124345,1.888244,1.971828
1,1,1.0,2.0,1.0,1.0,1.0,3.789688,4.682275,3.510520,3.011858
2,2,1.0,3.0,1.0,1.0,0.0,4.336946,2.471132,1.272698,1.540864
3,3,1.0,4.0,1.0,1.0,0.0,1.295495,1.456000,1.386428,1.160023
4,4,1.0,5.0,1.0,1.0,1.0,1.912048,2.154422,1.431473,1.264020
...,...,...,...,...,...,...,...,...,...,...
5495,5495,500.0,7.0,1.0,1.0,0.0,-0.289759,1.912800,-1.068690,0.615442
5496,5496,500.0,8.0,1.0,1.0,0.0,1.389520,0.695095,-0.697456,1.780840
5497,5497,500.0,9.0,1.0,1.0,0.0,0.389651,0.817572,-0.199511,1.429284
5498,5498,500.0,10.0,1.0,1.0,0.0,0.029739,-0.598246,-0.371662,0.518159


We now read the trajectory from the tabular format into Trajectory Arrays.

In [4]:
zs, states, actions, rewards, ids = read_trajectory_from_dataframe(
                                                data=trajectory, 
                                                z_labels=['z1', 'z2'], 
                                                state_labels=['state1', 'state2', 'state3'], 
                                                action_label='action', 
                                                reward_label='reward', 
                                                id_label='ID', 
                                                T=10
                                                )

## Preprocessor Training

Before preprocessing the trajectory, we need to first train a preprocessor. To mitigate overfitting, we use a random subset of 250 individuals in the trajectory to train the preprocessor. The remaining 250 individuals will be actually preprocessed. We now form these two sets.

In [5]:
(
    zs_train, zs_prepro, 
    states_train, states_prepro, 
    actions_train, actions_prepro, 
    rewards_train, rewards_prepro, 
    ids_train, ids_prepro
) = train_test_split(zs, states, actions, rewards, ids, test_size=0.5)

We now use the training set to train a `SequentialPreprocessor`.

In [6]:
sp = SequentialPreprocessor(z_space=[[0, 0], [0, 1], [1, 0], [1, 1]], 
                            num_actions=2, 
                            cross_folds=1, 
                            mode='single', 
                            reg_model='nn')
sp.train_preprocessor(zs=zs_train, xs=states_train, actions=actions_train, rewards=rewards_train)

100%|██████████| 1000/1000 [00:34<00:00, 28.59it/s]


(array([[[ 1.47487332e-01,  1.50254830e+00, -1.66469333e+00, ...,
           2.05579699e+00,  3.49205066e+00,  5.78904901e-01],
         [ 1.54685410e-01,  2.75040511e-01, -1.49451782e+00, ...,
           2.73385984e+00,  2.65295518e+00,  6.04489106e-01],
         [-1.32980964e+00, -1.11267430e+00, -1.99076036e+00, ...,
           2.85810735e+00,  6.95762332e-01, -4.31892000e-02],
         ...,
         [-5.66250520e-02, -8.48483132e-01, -1.17231567e+00, ...,
           4.76362781e-01,  3.89372482e-01, -8.59598406e-01],
         [-1.97910154e+00, -8.23400731e-01, -1.78218275e+00, ...,
          -1.01817249e-01,  4.91883402e-01, -3.73013399e-01],
         [-5.98881224e-01, -1.19249451e+00, -2.65340119e+00, ...,
           1.33478369e+00, -8.91307651e-01, -2.23343658e+00]],
 
        [[ 8.49102267e-01,  1.17672025e+00, -5.48338858e-01, ...,
           2.75741192e+00,  3.16622261e+00,  1.69525937e+00],
         [-1.93749297e+00,  2.51739130e-01, -2.68465126e+00, ...,
          -8.19620554

## Data Preprocessing

We now preprocess the remaining data that are not in the training set.

In [7]:
states_tilde, rewards_tilde = sp.preprocess_multiple_steps(zs=zs_prepro, 
                                                           xs=states_prepro, 
                                                           actions=actions_prepro, 
                                                           rewards=rewards_prepro)

## Data Exporting

Finally, we convert the preprocessed trajectory back into the tabular format so that it is easier to store and manage. For simplicity, we call the new states as `state1`, ..., `state12`, which is the default option provided by `convert_trajectory_to_dataframe()` (so we do not need to specify the `state_labels` argument).

In [8]:
preprocessed_trajectory = convert_trajectory_to_dataframe(
                                        zs=zs_prepro, 
                                        states=states_tilde, 
                                        actions=actions_prepro, 
                                        rewards=rewards_tilde, 
                                        ids=ids_prepro, 
                                        z_labels=['z1', 'z2'], 
                                        action_label='action', 
                                        reward_label='reward', 
                                        id_label='ID', 
                                        T_label='time_step'
                                        )
preprocessed_trajectory

,ID,time_step,z1,z2,action,reward,state1,state2,state3,state4,state5,state6,state7,state8,state9,state10,state11,state12
0,152.0,1.0,1.0,0.0,NaN,NaN,-0.017410,1.662376,-0.623004,0.802870,2.726212,0.471634,0.777608,2.598996,0.598366,1.890900,3.651878,1.620594
1,152.0,2.0,1.0,0.0,1.0,1.144948,0.393017,1.477447,-2.169418,1.541494,2.183505,-1.011502,1.015591,1.068420,-0.570223,4.529966,3.816629,1.264203
2,152.0,3.0,1.0,0.0,0.0,-0.547446,-1.652412,0.103718,-2.986552,-1.268881,1.023682,-2.484174,-2.133878,-0.034526,-3.075978,1.003205,2.167258,-0.910568
3,152.0,4.0,1.0,0.0,1.0,-0.655269,-0.684673,-0.986255,-2.821864,0.421235,-1.372006,-3.127629,-1.377760,-0.588429,-2.402069,1.736868,1.416677,-1.098827
4,152.0,5.0,1.0,0.0,1.0,-1.042717,-0.606745,-0.787610,-0.710801,-0.122448,-1.411133,-0.146238,-0.200898,-0.012166,-0.767219,2.383006,2.050712,1.159624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,278.0,7.0,0.0,1.0,0.0,-1.572233,-2.438056,1.066950,-1.844240,-1.420195,0.263312,-0.876003,-2.019137,-0.344875,-0.595312,-0.322164,0.653951,-0.409356
2746,278.0,8.0,0.0,1.0,1.0,-1.351713,-1.241231,-0.088648,-1.576157,-0.668799,0.403609,-1.427466,0.114344,-0.245085,-0.423337,1.814147,0.718364,0.275104
2747,278.0,9.0,0.0,1.0,0.0,-0.603052,-1.436617,0.053721,-0.074008,-0.945153,1.513795,0.285303,-1.459915,0.079338,0.510699,0.522905,1.504525,0.429474
2748,278.0,10.0,0.0,1.0,0.0,-0.242086,-2.408253,-0.620962,-0.440553,-1.379366,0.032526,0.005245,-1.534779,0.612330,0.434021,-0.892494,1.273294,0.596993


## Alternative: Preprocessing All Individuals

Sometimes, the number of individuals in the trajectory is small. In this case, if we only preprocess a subset of individuals, the resulting preprocessed trajectory might be too small to be useful for policy learning. In this case, we can directly preprocess all individuals using the `train_preprocessor()` function when we set `cross_folds` to a relatively large number.

When `cross_folds=K` where `K` is greater than 1, `train_preprocessor()` will internally divide the training data into `K` folds. For each $i=1,\dots,k$, it trains a transition dynamics model based on all the folds other than the $i$-th one, and this model is then used to preprocess data in the $i$-th fold. This results in `K` folds of preprocessed data, each of which is processed using a model that is trained on the other folds. These `K` folds of preprocessed data are then combined and returned by `train_preprocessor()`. This method allows us to preprocess all individuals in the trajectory while reducing overfitting.

To use this functionality, we first initialize a `SequentialPreprocessor` with `cross_folds` greater than 1. We use `cross_folds=5` here.

In [9]:
sp_cf5 = SequentialPreprocessor(z_space=[[0, 0], [0, 1], [1, 0], [1, 1]], 
                                num_actions=2, 
                                cross_folds=5, 
                                mode='single', 
                                reg_model='nn')

We now simultaneously train the preprocessor and preprocess all individuals in the trajectory using the precedure described above.

In [10]:
states_tilde_cf5, rewards_tilde_cf5 = sp_cf5.train_preprocessor(zs=zs, 
                                                                xs=states, 
                                                                actions=actions, 
                                                                rewards=rewards)

100%|██████████| 1000/1000 [00:51<00:00, 19.49it/s]


We can now convert the preprocessed trajectory into the tabular format.

In [11]:
preprocessed_trajectory_cf5 = convert_trajectory_to_dataframe(
                                            zs=zs, 
                                            states=states_tilde_cf5, 
                                            actions=actions, 
                                            rewards=rewards_tilde_cf5, 
                                            ids=ids, 
                                            z_labels=['z1', 'z2'], 
                                            action_label='action', 
                                            reward_label='reward', 
                                            id_label='ID', 
                                            T_label='time_step'
                                            )
preprocessed_trajectory_cf5

,ID,time_step,z1,z2,action,reward,state1,state2,state3,state4,state5,state6,state7,state8,state9,state10,state11,state12
0,1.0,1.0,1.0,1.0,NaN,NaN,2.110949,0.105924,0.026649,3.216404,1.071328,0.984715,3.075024,0.779309,1.069998,4.124345,1.888244,1.971828
1,1.0,2.0,1.0,1.0,1.0,2.929707,1.137483,0.598599,0.342339,2.661781,2.434062,1.371084,2.340262,2.343849,0.649790,4.682275,3.510520,3.011858
2,1.0,3.0,1.0,1.0,0.0,3.215733,-0.235605,-1.268949,0.269024,0.961248,0.166175,0.038145,0.944498,-0.096271,-0.112551,2.471132,1.272698,1.540864
3,1.0,4.0,1.0,1.0,0.0,0.442691,-0.475055,-0.487856,-0.203484,-0.050267,-0.021863,-0.247616,-0.352119,-0.631286,-0.748012,1.456000,1.386428,1.160023
4,1.0,5.0,1.0,1.0,1.0,1.061733,0.111948,-0.945874,-0.586603,0.633861,-0.275382,-0.718945,0.473692,-0.571664,-0.748064,2.154422,1.431473,1.264020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5495,500.0,7.0,1.0,1.0,0.0,-0.809426,-0.001833,-2.961463,-0.260486,1.103740,-3.577922,-0.120947,-0.011743,-1.914502,-0.596573,1.912800,-1.068690,0.615442
5496,500.0,8.0,1.0,1.0,0.0,0.458912,-1.202421,-1.283641,1.344324,-0.247207,-3.222864,1.325328,-1.019518,-1.059411,1.765072,0.695095,-0.697456,1.780840
5497,500.0,9.0,1.0,1.0,0.0,-0.114715,0.951564,-1.275389,0.004058,0.060901,-1.283716,-0.790676,0.999890,-0.394687,0.844166,0.817572,-0.199511,1.429284
5498,500.0,10.0,1.0,1.0,0.0,-0.338144,-0.282491,-1.903879,-0.615268,-0.322978,-1.861784,-1.317675,-0.386151,-1.601404,-0.885262,-0.598246,-0.371662,0.518159
